In [11]:
import tkinter as tk
from tkinter import simpledialog, messagebox

class Transaction:
    def __init__(self, transaction_type, amount, recipient=None):
        self.transaction_type = transaction_type
        self.amount = amount
        self.recipient = recipient
        self.timestamp = self.get_timestamp()

    def get_timestamp(self):
        import time
        return time.strftime("%Y-%m-%d %H:%M:%S")

class User:
    def __init__(self, user_id, pin):
        self.user_id = user_id
        self.pin = pin
        self.balance = 0
        self.transaction_history = []

    def deposit(self, amount):
        if amount > 0:
            self.balance += amount
            self.transaction_history.append(Transaction("Deposit", amount))
            print(f"Deposit of ${amount} is successful.")
        else:
            print("Please enter a valid deposit amount (positive value).")

    def withdraw(self, amount):
        if self.balance >= amount:
            self.balance -= amount
            self.transaction_history.append(Transaction("Withdraw", amount))
            print(f"Withdrawal of ${amount} is successful.")
        else:
            print("Insufficient balance.")

    def transfer(self, recipient, amount):
        if self.balance >= amount:
            self.balance -= amount
            self.transaction_history.append(Transaction("Transfer", amount, recipient.user_id))
            print(f"Transfer of ${amount} to {recipient.user_id} is successful.")
        else:
            print("Insufficient balance.")

    def view_transaction_history(self):
        for transaction in self.transaction_history:
            recipient_info = f" to {transaction.recipient}" if transaction.recipient else ""
            print(f"{transaction.timestamp}: {transaction.transaction_type} - {transaction.amount}{recipient_info}")

class ATM(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("ATM")
        self.geometry("400x300")

        self.users = {
            "user1": User("user1", "1111"),
            "user2": User("user2", "2222"),
            "user3": User("user3", "3333"),
        }
        self.current_user = None

        self.create_widgets()
    
    def create_widgets(self):
        self.label_user_id = tk.Label(self, text="User ID:")
        self.label_user_id.pack()
        self.entry_user_id = tk.Entry(self)
        self.entry_user_id.pack()

        self.label_pin = tk.Label(self, text="PIN:")
        self.label_pin.pack()
        self.entry_pin = tk.Entry(self, show="*")
        self.entry_pin.pack()

        self.button_login = tk.Button(self, text="Login", command=self.login)
        self.button_login.pack()

        self.button_signup = tk.Button(self, text="Sign Up", command=self.signup)
        self.button_signup.pack()

        self.label_message = tk.Label(self, text="")
        self.label_message.pack()

        self.menu_frame = tk.Frame(self)

        self.button_deposit = tk.Button(self.menu_frame, text="Deposit", command=self.deposit)
        self.button_deposit.pack()

        self.button_withdraw = tk.Button(self.menu_frame, text="Withdraw", command=self.withdraw)
        self.button_withdraw.pack()

        self.button_transfer = tk.Button(self.menu_frame, text="Transfer", command=self.transfer)
        self.button_transfer.pack()

        self.button_transaction_history = tk.Button(self.menu_frame, text="Transaction History", command=self.transaction_history)
        self.button_transaction_history.pack()

        self.button_check_balance = tk.Button(self.menu_frame, text="Check Balance", command=self.check_balance)
        self.button_check_balance.pack()

        self.button_exit = tk.Button(self.menu_frame, text="Exit", command=self.exit)
        self.button_exit.pack()

    def login(self):
        user_id = self.entry_user_id.get()
        pin = self.entry_pin.get()

        if user_id in self.users and self.users[user_id].pin == pin:
            self.current_user = self.users[user_id]
            self.label_message.config(text="Login successful.")
            self.entry_user_id.delete(0, tk.END)
            self.entry_pin.delete(0, tk.END)
            self.menu_frame.pack()
            self.show_withdraw_page()  # Redirect to withdrawal page
        else:
            self.label_message.config(text="Invalid User ID or PIN. Please try again.")

    def signup(self):
        user_id = simpledialog.askstring("Sign Up", "Enter User ID:")
        pin = simpledialog.askstring("Sign Up", "Enter PIN:")
        if user_id and pin:
            self.users[user_id] = User(user_id, pin)
            messagebox.showinfo("Sign Up", "Sign up successful. You can now log in.")

    def show_withdraw_page(self):
        self.menu_frame.tkraise()

    def deposit(self):
        amount = float(simpledialog.askstring("Deposit", "Enter the deposit amount: $"))
        if amount:
            self.current_user.deposit(amount)

    def withdraw(self):
        amount = float(simpledialog.askstring("Withdraw", "Enter the withdrawal amount: $"))
        if amount:
            self.current_user.withdraw(amount)

    def transfer(self):
        recipient_id = simpledialog.askstring("Transfer", "Enter recipient's User ID:")
        if recipient_id in self.users:
            amount = float(simpledialog.askstring("Transfer", "Enter the transfer amount: $"))
            if amount:
                recipient = self.users[recipient_id]
                self.current_user.transfer(recipient, amount)
        else:
            messagebox.showerror("Error", "Recipient not found.")

    def transaction_history(self):
        if self.current_user:
            messagebox.showinfo("Transaction History", "\n".join([f"{transaction.timestamp}: {transaction.transaction_type} - {transaction.amount}" for transaction in self.current_user.transaction_history]))
        else:
            messagebox.showerror("Error", "Please log in first.")

    def check_balance(self):
        if self.current_user:
            messagebox.showinfo("Balance", f"Account Balance: ${self.current_user.balance}")
        else:
            messagebox.showerror("Error", "Please log in first.")

    def exit(self):
        self.destroy()

if __name__ == "__main__":
    app = ATM()
    app.mainloop()
